In [57]:
import numpy as np
import json
import sys
import requests
import random
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
from typing import List, Dict

from CrawlerManager import CrawlerManager
from post_processing import main_process

chrome_options = Options()
chrome_options.add_argument("--headless")
gl_SPOOFED_USER_AGENT = ("Mozilla/5.0 (X11; Linux x86_64) "
                         "AppleWebKit/537.36 "
                         "(KHTML, like Gecko) Chrome/70.0.3538.77 "
                         "Safari/537.36")
chrome_options.add_argument("--user-agent=%s" %
                            gl_SPOOFED_USER_AGENT)
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("ignore-certificate-errors")
chrome_options.add_argument("incognito")
chrome_options.add_argument("disable-gpu")
chrome_options.add_argument("disable-xss-auditor")
# chrome_options.add_argument("disable-background-networking")
chrome_options.add_argument("mute-audio")
chrome_options.add_argument("disable-notifications")
chrome_options.add_argument("allow-running-insecure-content")

In [2]:
from collect_cookies import *

In [65]:
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://duo.nl')
driver.get('https://nwo.nl')
cookies = driver.execute_cdp_cmd('Network.getAllCookies', dict())['cookies']

len(cookies), cookies

(6,
 [{'domain': 'duo.nl',
   'expires': -1,
   'httpOnly': True,
   'name': 'JSESSIONID',
   'path': '/',
   'priority': 'Medium',
   'sameParty': False,
   'secure': True,
   'session': True,
   'size': 42,
   'sourcePort': 443,
   'sourceScheme': 'Secure',
   'value': '63E11D66EB78DAF16C462DBEA224233E'},
  {'domain': '.duo.nl',
   'expires': 1647993819,
   'httpOnly': False,
   'name': '_pk_ses.1.0569',
   'path': '/',
   'priority': 'Medium',
   'sameParty': False,
   'secure': False,
   'session': False,
   'size': 15,
   'sourcePort': 443,
   'sourceScheme': 'Secure',
   'value': '*'},
  {'domain': '.duo.nl',
   'expires': 1681947219,
   'httpOnly': False,
   'name': '_pk_id.1.0569',
   'path': '/',
   'priority': 'Medium',
   'sameParty': False,
   'secure': False,
   'session': False,
   'size': 65,
   'sourcePort': 443,
   'sourceScheme': 'Secure',
   'value': '8ef26ecd05881b3d.1647992019.1.1647992020.1647992019.'},
  {'domain': '.nwo.nl',
   'expires': 1711064021,
   'httpOnl

In [68]:
def get_parse_cookies(urls: List[str], website_name: str, fields: List[str]) -> List[Dict]:
    # Create new driver because it will otherwise return all the cookies.
    driver = webdriver.Chrome(options=chrome_options)
    # Create new driver because it will otherwise return all the cookies.
    for url in urls:
        print(url)
        driver.get(url)
    # Empty dict for the input arguments.
    cookies = driver.execute_cdp_cmd('Network.getAllCookies', dict())['cookies']
    wanted_data = []
    for c in cookies:
        cookie_dict = {k: c[k] for k in fields}
        cookie_dict['third_party'] = website_name not in cookie_dict['domain']
        cookie_dict['trackers_list'] = check_trackers(cookie_dict['domain'])
        wanted_data.append(cookie_dict)

    # print(url)
    return wanted_data

In [5]:
url = 'https://business.gov.nl/'

In [66]:
def hop(base_url: str) -> List[str]:
    r = requests.get(base_url)
    soup = BeautifulSoup(r.content, 'html.parser')
    a_tags = soup.find_all('a', href=True)
    # Create absolute URLs and remove the ones which are refs on the same page.
    w_name = get_website_name(base_url)
    refs = {full_url for x in a_tags if w_name == get_website_name(full_url := urljoin(base_url, x['href'])) and not x['href'].startswith('#')}
    return random.sample([*refs], min(len(refs), 20))

drollies = hop(url)

In [69]:
fields = ['name', 'domain', 'expires']
get_parse_cookies(drollies, get_website_name(url), fields)

https://business.gov.nl/running-your-business/business-location/
https://business.gov.nl/webinars-and-videos/how-to-start-an-online-shop-in-the-netherlands/webinar-on-demand/
https://business.gov.nl/starting-your-business/choosing-a-business-structure/
https://business.gov.nl/partners/netherlands-chamber-of-commerce/
https://business.gov.nl/running-your-business/business-management/insurance/
https://business.gov.nl/privacy-cookies/
https://business.gov.nl/regulation/leave-schemes/
https://business.gov.nl/coming-to-the-netherlands/information-for-employers/
https://business.gov.nl/
https://business.gov.nl/disclaimer/
https://business.gov.nl/running-your-business/international-business/
https://business.gov.nl/overview/
https://business.gov.nl/search
https://business.gov.nl/financing-your-business/subsidies/
https://business.gov.nl/financing-your-business/funding-and-loans/
https://business.gov.nl/running-your-business/business-taxes/
https://business.gov.nl/webinars-and-videos/financin

[{'name': '_gid',
  'domain': '.gov.nl',
  'expires': 1648079275,
  'third_party': False,
  'trackers_list': []},
 {'name': '_gat_UA-3742073-10',
  'domain': '.gov.nl',
  'expires': 1647992888,
  'third_party': False,
  'trackers_list': []},
 {'name': '_hjFirstSeen',
  'domain': '.gov.nl',
  'expires': 1647994629,
  'third_party': False,
  'trackers_list': []},
 {'name': '_hjIncludedInSessionSample',
  'domain': 'business.gov.nl',
  'expires': 1647992995,
  'third_party': False,
  'trackers_list': []},
 {'name': '_hjSession_630207',
  'domain': '.gov.nl',
  'expires': 1647994675,
  'third_party': False,
  'trackers_list': []},
 {'name': '_hjAbsoluteSessionInProgress',
  'domain': '.gov.nl',
  'expires': 1647994675,
  'third_party': False,
  'trackers_list': []},
 {'name': '_hjSessionUser_630207',
  'domain': '.gov.nl',
  'expires': 1679528875,
  'third_party': False,
  'trackers_list': []},
 {'name': '_gat_UA-3742073-13',
  'domain': '.gov.nl',
  'expires': 1647992894,
  'third_party':

In [70]:
get_parse_cookies(['https://business.gov.nl'], 'gov', fields)

https://business.gov.nl


[{'name': '_ga_845PX04177',
  'domain': '.gov.nl',
  'expires': 1711064999,
  'third_party': False,
  'trackers_list': []},
 {'name': '_ga',
  'domain': '.gov.nl',
  'expires': 1711064999,
  'third_party': False,
  'trackers_list': []},
 {'name': '_gid',
  'domain': '.gov.nl',
  'expires': 1648079399,
  'third_party': False,
  'trackers_list': []},
 {'name': '_gat_UA-3742073-10',
  'domain': '.gov.nl',
  'expires': 1647993059,
  'third_party': False,
  'trackers_list': []},
 {'name': '_hjSessionUser_630207',
  'domain': '.gov.nl',
  'expires': 1679528999,
  'third_party': False,
  'trackers_list': []},
 {'name': '_hjFirstSeen',
  'domain': '.gov.nl',
  'expires': 1647994799,
  'third_party': False,
  'trackers_list': []},
 {'name': '_hjIncludedInSessionSample',
  'domain': 'business.gov.nl',
  'expires': 1647993119,
  'third_party': False,
  'trackers_list': []},
 {'name': '_hjSession_630207',
  'domain': '.gov.nl',
  'expires': 1647994799,
  'third_party': False,
  'trackers_list': []

In [8]:
d = {'abc.com': {'frontpage': 'cookie1', 'hopped': ['cookie2', 'cookie3']}}

with open('drol.json', 'w') as f:
    json.dump(d, f, indent=2)